## TensorFlow: Convolutional NN
#### Components Of The Model

- Model Function (```cnn_model_fn```)
 - accept features, class labels, mode and model params as args
 - define the layers
 - define a dictionary for output of predictions
 - create `EstimatorSpec` object for the appropriate mode
   - train, predict, eval
   - create one-hot from class labels for train and eval
   - eval needs dict of metrics to use
- Main Function (`main`)
 - accept mode and model params as args
 - call a function to get data
   - load MNIST from TF in this case
 - create the estimator with `cnn_model_fn` and model params
 - create `*_input_fn` where `*` is the mode (e.g. train)
   - uses `numpy_input_fn` from the TF API for numpy data
 - run the classifier/estimator using the appropriate mode
 - perform any work necessary to display or return results

In [1]:
import numpy as np
import tensorflow as tf

D:\Users\snandan.ORADEV\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)


## Model Function
def cnn_model_fn(features, labels, mode, params):
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    # MNIST images are 28x28 pixels, and have one color channel
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Computes 32 features using a 5x5 filter with ReLU activation
    # Input Tensor Shape: [batch_size, 28, 28, 1]
    # Output Tensor Shape: [batch_size, 28, 28, 32]
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",  # use "valid" to not preserve WxH
        activation=tf.nn.relu)

    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 28, 28, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Computes 64 feature maps using a 5x5 filter.
    # Input Tensor Shape: [batch_size, 14, 14, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 64]
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Input Tensor Shape: [batch_size, 14, 14, 64]
    # Output Tensor Shape: [batch_size, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Flatten tensor into a batch of vectors for input to dense layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

    # Input Tensor Shape: [batch_size, 7 * 7 * 64]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout, units=10)

    # this dict will be returned for predictions
    predictions = {
        # actual class predictions
        "classes": tf.argmax(input=logits, axis=1),
        # class probabilities from softmax on logits
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

    # Configure the training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=params['learning_rate'])
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    


## Main Function
def main(mode='train', model_params={'learning_rate': 0.001}):
    # Load training and eval data
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images  # Returns np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images  # Returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

    # Create the Estimator
    mnist_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn,
        params=model_params,
        model_dir="./tmp/mnist_convnet_model")

    # Train the model
    if mode == 'train':
        train_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": train_data},
            y=train_labels,
            batch_size=100,
            num_epochs=10,
            shuffle=True)
        mnist_classifier.train(
            input_fn=train_input_fn)
    elif mode == 'predict':
        predict_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": eval_data},
            num_epochs=1,
            shuffle=False)
        preds = mnist_classifier.predict(
            input_fn=predict_input_fn)
        return np.array([p for p in preds])
    elif mode == 'eval':
        # Evaluate the model and print results
        eval_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": eval_data},
            y=eval_labels,
            num_epochs=1,
            shuffle=False)
        eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
        print(eval_results)

In [3]:
main()

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data\t10k-labels-idx1-u

INFO:tensorflow:loss = 0.4840279, step = 3101 (171.658 sec)
INFO:tensorflow:Saving checkpoints for 3117 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:global_step/sec: 0.58701
INFO:tensorflow:loss = 0.46915904, step = 3201 (170.356 sec)
INFO:tensorflow:global_step/sec: 0.58259
INFO:tensorflow:loss = 0.37749836, step = 3301 (171.646 sec)
INFO:tensorflow:global_step/sec: 0.588978
INFO:tensorflow:loss = 0.4142326, step = 3401 (169.786 sec)
INFO:tensorflow:Saving checkpoints for 3469 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:global_step/sec: 0.584696
INFO:tensorflow:loss = 0.57717454, step = 3501 (171.029 sec)
INFO:tensorflow:global_step/sec: 0.574602
INFO:tensorflow:loss = 0.5902504, step = 3601 (174.033 sec)
INFO:tensorflow:global_step/sec: 0.55767
INFO:tensorflow:loss = 0.38206422, step = 3701 (179.309 sec)
INFO:tensorflow:global_step/sec: 0.553064
INFO:tensorflow:loss = 0.25487447, step = 3801 (180.820 sec)
INFO:tensorflow:Saving checkpoints for 3807 into /tm

In [4]:
main(mode='eval')

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000215CCF794E0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 20

In [5]:
preds = main(mode='predict')

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000215CBB743C8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO

In [6]:
preds[0]

{'classes': 7,
 'probabilities': array([1.5606147e-05, 7.4664865e-07, 5.3925633e-06, 2.1341136e-04,
        1.9175222e-06, 1.6620263e-05, 3.9442714e-08, 9.9903119e-01,
        1.8132181e-05, 6.9692050e-04], dtype=float32)}